In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [17]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Merge, Embedding, Convolution1D, Dropout, Bidirectional
from attention_helper import *

from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence


from features import *
from helper import *

In [10]:
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = '/media/bagus/data01/dataset/IEMOCAP_full_release/'
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000

In [11]:
import pickle
with open('/media/bagus/data01/dataset/IEMOCAP_full_release/data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [12]:
text = []

for ses_mod in data2:
    text.append(ses_mod['transcription'])

In [13]:
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [14]:
import codecs
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

file_loc = '../../data/glove.840B.300d.txt'

print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 2736 unique tokens
../../data/glove.840B.300d.txt
G Word embeddings: 2196018
G Null word embeddings: 100


In [10]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4936, 4)

In [15]:
model = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model.add(Convolution1D(256, 3, padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(128, 3, padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(64, 3, padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(32, 3, padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu')) 
model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

#model.compile()
model.summary()

print("Model1 Built")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 256)          230656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 256)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 500, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 128)          98432     
_______________________________________________________

In [9]:
hist = model.fit(x_train_text, Y, 
                 batch_size=100, epochs=125, verbose=1, 
                 validation_split=0.2)

/media/bagus/data01/github/IEMOCAP-Emotion-Detection/er/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/125
3948/3948 [==============================] - 17s - loss: 1.3682 - acc: 0.3437 - val_loss: 1.3326 - val_acc: 0.3897
Epoch 2/125
3948/3948 [==============================] - 16s - loss: 1.3042 - acc: 0.3713 - val_loss: 1.2245 - val_acc: 0.4231
Epoch 3/125
3948/3948 [==============================] - 16s - loss: 1.1635 - acc: 0.4607 - val_loss: 1.1442 - val_acc: 0.4828
Epoch 4/125
3948/3948 [==============================] - 16s - loss: 0.9722 - acc: 0.5798 - val_loss: 1.0299 - val_acc: 0.5840
Epoch 5/125
3948/3948 [==============================] - 16s - loss: 0.7775 - acc: 0.6811 - val_loss: 0.9788 - val_acc: 0.6316
Epoch 6/125
3948/3948 [==============================] - 16s - loss: 0.6349 - acc: 0.7543 - val_loss: 1.0378 - val_acc: 0.6174
Epoch 7/125
3948/3948 [==============================] - 16s - loss: 0.5266 - acc: 0.7936 - val_loss: 1.1187 - val_acc: 0.6053
Epoch 8/125
3948/3948 [==============================] - 16s - l

In [10]:
acc1 = max(hist.history['val_acc'])
print(acc1)

0.65688259012786


In [60]:
model = Sequential()
model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))

model.add(LSTM(512, return_sequences=True))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

#model.compile()
model.summary()

print("Model2 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 500, 300)          821100    
_________________________________________________________________
lstm_71 (LSTM)               (None, 500, 512)          1665024   
_________________________________________________________________
lstm_72 (LSTM)               (None, 256)               787456    
_________________________________________________________________
dense_37 (Dense)             (None, 512)               131584    
_________________________________________________________________
activation_37 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 4)                 2052      
_________________________________________________________________
activation_38 (Activation)   (None, 4)                 0         
Total para

In [53]:
hist = model.fit(x_train_text, Y, 
                 batch_size=16, epochs=125, verbose=1, 
                 validation_split=0.2)

Train on 3948 samples, validate on 988 samples
Epoch 1/125
3948/3948 [==============================] - 241s - loss: 1.3100 - acc: 0.3954 - val_loss: 1.1864 - val_acc: 0.4626
Epoch 2/125
3948/3948 [==============================] - 238s - loss: 1.0652 - acc: 0.5484 - val_loss: 1.0054 - val_acc: 0.6032
Epoch 3/125
3948/3948 [==============================] - 238s - loss: 0.8480 - acc: 0.6631 - val_loss: 1.0126 - val_acc: 0.5749
Epoch 4/125
3948/3948 [==============================] - 238s - loss: 0.7060 - acc: 0.7239 - val_loss: 0.9911 - val_acc: 0.6225
Epoch 5/125
3948/3948 [==============================] - 238s - loss: 0.5972 - acc: 0.7573 - val_loss: 0.9178 - val_acc: 0.6508
Epoch 6/125
3948/3948 [==============================] - 238s - loss: 0.5017 - acc: 0.8060 - val_loss: 0.9980 - val_acc: 0.6387
Epoch 7/125
3948/3948 [==============================] - 238s - loss: 0.4360 - acc: 0.8341 - val_loss: 1.1513 - val_acc: 0.6306
Epoch 8/125
3948/3948 [==============================] - 

In [54]:
acc2=max(hist.history['val_acc'])
print(acc2)

0.651821860176349


In [14]:
len(tokenizer.word_index)

2736

In [58]:
model = Sequential()
model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512, return_sequences=False))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(4))
model.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

print("Model3 Built")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 500, 300)          821100    
_________________________________________________________________
lstm_62 (LSTM)               (None, 500, 512)          1665024   
_________________________________________________________________
lstm_63 (LSTM)               (None, 500, 512)          2099200   
_________________________________________________________________
lstm_64 (LSTM)               (None, 500, 512)          2099200   
_________________________________________________________________
lstm_65 (LSTM)               (None, 500, 512)          2099200   
_________________________________________________________________
lstm_66 (LSTM)               (None, 500, 512)          2099200   
_________________________________________________________________
lstm_67 (LSTM)               (None, 500, 512)          2099200   
__________

In [56]:
hist = model.fit(x_train_text, Y, 
                 batch_size=64, epochs=125, verbose=1, 
                 validation_split=0.2)

Train on 3948 samples, validate on 988 samples
Epoch 1/100
3948/3948 [==============================] - 518s - loss: 1.3810 - acc: 0.3402 - val_loss: 1.3699 - val_acc: 0.3573
Epoch 2/100
3948/3948 [==============================] - 514s - loss: 1.3733 - acc: 0.3468 - val_loss: 1.3463 - val_acc: 0.3968
Epoch 3/100
3948/3948 [==============================] - 510s - loss: 1.3744 - acc: 0.3506 - val_loss: 1.3641 - val_acc: 0.3694
Epoch 4/100
3948/3948 [==============================] - 510s - loss: 1.3681 - acc: 0.3503 - val_loss: 1.3530 - val_acc: 0.3846
Epoch 5/100
3948/3948 [==============================] - 510s - loss: 1.3298 - acc: 0.3944 - val_loss: 1.3535 - val_acc: 0.3785
Epoch 6/100
3948/3948 [==============================] - 510s - loss: 1.2460 - acc: 0.4276 - val_loss: 1.2699 - val_acc: 0.4443
Epoch 7/100
3948/3948 [==============================] - 510s - loss: 1.1412 - acc: 0.4934 - val_loss: 1.2595 - val_acc: 0.4575
Epoch 8/100
3948/3948 [==============================] - 

In [57]:
acc3=max(hist.history['val_acc'])
print(acc3)

0.6417004002733269


In [23]:
# Try BLSTM with attention model

def attention_model(optimizer='rmsprop'):
    model = Sequential()
    model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
    model.add(LSTM(128, return_sequences=True))
    model.add(AttentionDecoder(128, 128))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

model4 = attention_model()
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
lstm_4 (LSTM)                (None, 500, 128)          219648    
_________________________________________________________________
AttentionDecoder (AttentionD (None, 500, 128)          246528    
_________________________________________________________________
flatten_4 (Flatten)          (None, 64000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               32768512  
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 2052      
Total params: 34,057,840
Trainable params: 34,057,840
Non-trainable params: 0
________________________________________________________________

In [24]:
hist4 = model4.fit(x_train_text, Y, batch_size=64, epochs=125, verbose=1, validation_split=0.2)

Train on 3948 samples, validate on 988 samples
Epoch 1/25
3948/3948 [==============================] - 854s - loss: 1.5641 - acc: 0.3070 - val_loss: 1.4088 - val_acc: 0.3694
Epoch 2/25
3948/3948 [==============================] - 847s - loss: 1.4590 - acc: 0.2966 - val_loss: 1.5173 - val_acc: 0.1630
Epoch 3/25
3948/3948 [==============================] - 843s - loss: 1.4092 - acc: 0.3625 - val_loss: 1.3225 - val_acc: 0.3472
Epoch 4/25
3948/3948 [==============================] - 843s - loss: 1.1294 - acc: 0.5233 - val_loss: 1.0990 - val_acc: 0.5729
Epoch 5/25
3948/3948 [==============================] - 854s - loss: 0.9794 - acc: 0.6102 - val_loss: 0.9835 - val_acc: 0.5992
Epoch 6/25
3948/3948 [==============================] - 805s - loss: 0.8652 - acc: 0.6535 - val_loss: 0.9351 - val_acc: 0.6326
Epoch 7/25
3948/3948 [==============================] - 803s - loss: 0.7666 - acc: 0.7097 - val_loss: 1.0101 - val_acc: 0.6437
Epoch 8/25
3948/3948 [==============================] - 1076s - 

In [26]:
acc4=max(hist4.history['val_acc'])
print(acc4)

0.6609311743303832
